In [ ]:
# /// script
# requires-python = ">=3.13"
# dependencies = [
#     "boto3",
#     "httpx",
#     "pystac-client",
# ]
# ///

# Update version metadata in the original icesat2-boreal collection

The goal is to add a `deprecated` tag to the collection metadata and provide a link to the latest version (icesat2-boreal-v2.1-agb).

In [2]:
import json

import boto3
import httpx
import pystac
import pystac_client
from pystac.extensions.version import VersionRelType


pystac.set_stac_version("1.0.0")


COLLECTION_ID = "icesat2-boreal"
VERSION = "v1.0"

# STAC ingestory URL
INGESTOR_URL = "https://stac-ingestor.maap-project.org"

## Step 1: get token for STAC ingestor API
This needs to be run with credentials for the SMCE MAAP AWS account set in the environment.

In [3]:
# paste MAAP SMCE AWS credentials here:
session = boto3.Session(
    region_name="us-west-2",
)
client = session.client("secretsmanager", region_name="us-west-2")

# MAAP STAC secret
response = client.get_secret_value(
    SecretId="arn:aws:secretsmanager:us-west-2:916098889494:secret:MAAP-STAC-auth-dev/MAAP-workflows-EsykqB"
)

settings = json.loads(response["SecretString"])

# function to get token for STAC ingestor
def get_token(
    client_id: str, 
    client_secret: str, 
    domain: str,
    scope: str
) -> str:
    response = httpx.post(
        f"{domain}/oauth2/token",
        headers={
            "Content-Type": "application/x-www-form-urlencoded",
        },
        auth=(client_id, client_secret),
        data={
            "grant_type": "client_credentials",
            "scope": scope,
        },
    )
    try:
        response.raise_for_status()
    except Exception:
        raise

    return response.json()["access_token"]


token = get_token(
    client_id = settings["client_id"],
    client_secret = settings["client_secret"],
    domain = settings["cognito_domain"],
    scope = settings["scope"],
)

## Step 2: retrieve the collection from the catalog

In [3]:
client = pystac_client.Client.open("https://stac.maap-project.org")
collection = client.get_collection(COLLECTION_ID)

## Step 3: add the version metadata
1. tag as v1.0
2. add link to latest version of the collection

In [4]:
# add version info
collection.ext.add("version")
collection.ext.version.version = VERSION
collection.ext.version.deprecated = True

# clean up links (because pgstac will add them automatically)
collection.links = [link for link in collection.links if link.rel == "root"]
collection.add_links(
    [
        pystac.Link(
            rel=VersionRelType.LATEST,
            target="https://stac.maap-project.org/collections/icesat2-boreal-v2.1-agb",
            title="version 2.1 (biomass)",
        ),
    ]
)


print(
    json.dumps(collection.to_dict(), indent=2)
)

{
  "type": "Collection",
  "id": "icesat2-boreal",
  "stac_version": "1.0.0",
  "description": "Aboveground biomass density c.2020 gridded to 30m derived from ICESat-2, Harmonized Landsat-Sentinel2 and Copernicus DEM. Band 1 is the data band. Band 2 is the standard deviation.",
  "links": [
    {
      "rel": "root",
      "href": "https://stac.maap-project.org",
      "type": "application/json",
      "title": "MAAP STAC API (dev)"
    },
    {
      "rel": "latest-version",
      "href": "https://stac.maap-project.org/collections/icesat2-boreal-v2.1-agb",
      "title": "version 2.1 (biomass)"
    }
  ],
  "stac_extensions": [
    "https://stac-extensions.github.io/render/v1.0.0/schema.json",
    "https://stac-extensions.github.io/item-assets/v1.0.0/schema.json",
    "https://stac-extensions.github.io/version/v1.2.0/schema.json"
  ],
  "renders": {
    "agb": {
      "bidx": [
        1
      ],
      "title": "ICESat-2 Boreal Biomass",
      "assets": [
        "tif"
      ],
     

## Step 4: Post to the STAC ingestor API

In [5]:
post_annual = httpx.post(
    f"{INGESTOR_URL}/collections",
    json=collection.to_dict(),
    headers = {
        'Authorization': f'Bearer {token}',
        'Content-Type': 'application/json',  # Assuming you are sending JSON data
    }
)
print(post_annual.json())

['Successfully published: icesat2-boreal']
